In [1]:

import pandas as pd 
import numpy as np
import requests
import io
import csv
from path import Path
import psycopg2
#from config import db_password

CREATE DATASET

In [2]:

# Connction to AWS nba database
connection = psycopg2.connect(
    host = 'nba-db.cpmpsi1pfaz0.us-east-2.rds.amazonaws.com',
    port = 5432,
    user = 'postgres',
    password = 'caf3rac3',
    database='postgres'
    )
cursor=connection.cursor()

In [3]:
#read in data from database for All NBA table
nba_unpivoted_sql = 'SELECT * FROM public.allnba_unpivoted'
All_NBA_Unpivoted_df = pd.read_sql(nba_unpivoted_sql, con=connection).drop(['Lg'],axis=1)
#rename the players column
All_NBA_Unpivoted_df=All_NBA_Unpivoted_df.rename(columns = {'Value':'PlayerName'})
#create column with the start year
All_NBA_Unpivoted_df['Season_Start_Year'] = All_NBA_Unpivoted_df['Season'].apply(lambda row: row[0:4])
All_NBA_Unpivoted_df['Season_Start_Year'] = pd.to_numeric(All_NBA_Unpivoted_df['Season_Start_Year'])
All_NBA_Unpivoted_df.head()

,Season,Tm,Position,PlayerName,Season_Start_Year
0,2019-20,1st,C,Anthony Davis,2019
1,2019-20,1st,F,Giannis Antetokounmpo,2019
2,2019-20,1st,F,LeBron James,2019
3,2019-20,1st,G,James Harden,2019
4,2019-20,1st,G,Luka Dončić,2019


In [4]:

#cleaning up the names with crazy characters
All_NBA_Unpivoted_df['PlayerName_Formatted'] = All_NBA_Unpivoted_df['PlayerName'].apply(lambda row: row.replace("ć","c"))
All_NBA_Unpivoted_df['PlayerName_Formatted'] = All_NBA_Unpivoted_df['PlayerName_Formatted'].apply(lambda row: row.replace("č","c"))
All_NBA_Unpivoted_df['PlayerName_Formatted'] = All_NBA_Unpivoted_df['PlayerName_Formatted'].apply(lambda row: row.replace("ž","z"))

#make dummies for position and team (1st, 2nd, 3rd)
All_NBA_Unpivoted_df = pd.get_dummies(All_NBA_Unpivoted_df,columns=['Position','Tm']).drop(['Season','PlayerName'],axis=1)
#rename the columns (cosmetic)
All_NBA_Unpivoted_df=All_NBA_Unpivoted_df.rename(columns = {'Position_C':'AllNBA_Center'})
All_NBA_Unpivoted_df=All_NBA_Unpivoted_df.rename(columns = {'Position_F':'AllNBA_Forward'})
All_NBA_Unpivoted_df=All_NBA_Unpivoted_df.rename(columns = {'Position_G':'AllNBA_Guard'})
All_NBA_Unpivoted_df.head(10)

,Season_Start_Year,PlayerName_Formatted,AllNBA_Center,AllNBA_Forward,AllNBA_Guard,Tm_1st,Tm_2nd,Tm_3rd
0,2019,Anthony Davis,1,0,0,1,0,0
1,2019,Giannis Antetokounmpo,0,1,0,1,0,0
2,2019,LeBron James,0,1,0,1,0,0
3,2019,James Harden,0,0,1,1,0,0
4,2019,Luka Doncic,0,0,1,1,0,0
5,2019,Nikola Jokic,1,0,0,0,1,0
6,2019,Kawhi Leonard,0,1,0,0,1,0
7,2019,Pascal Siakam,0,1,0,0,1,0
8,2019,Damian Lillard,0,0,1,0,1,0
9,2019,Chris Paul,0,0,1,0,1,0


In [5]:
#get list of all of the players that have made an all nba team
AllNBAPlayersList = All_NBA_Unpivoted_df['PlayerName_Formatted'].unique()
AllNBAPlayersList

array(['Anthony Davis', 'Giannis Antetokounmpo', 'LeBron James',
       'James Harden', 'Luka Doncic', 'Nikola Jokic', 'Kawhi Leonard',
       'Pascal Siakam', 'Damian Lillard', 'Chris Paul', 'Rudy Gobert',
       'Jayson Tatum', 'Jimmy Butler', 'Ben Simmons', 'Russell Westbrook',
       'Paul George', 'Stephen Curry', 'Joel Embiid', 'Kevin Durant',
       'Kyrie Irving', 'Blake Griffin', 'Kemba Walker',
       'LaMarcus Aldridge', 'DeMar DeRozan', 'Karl-Anthony Towns',
       'Victor Oladipo', 'Isaiah Thomas', 'DeAndre Jordan',
       'Draymond Green', 'John Wall', 'DeMarcus Cousins',
       'Andre Drummond', 'Klay Thompson', 'Kyle Lowry', 'Marc Gasol',
       'Pau Gasol', 'Tim Duncan', 'Joakim Noah', 'Dwight Howard',
       'Kevin Love', 'Tony Parker', 'Al Jefferson', 'Goran Dragic',
       'Kobe Bryant', 'Carmelo Anthony', 'David Lee', 'Dwyane Wade',
       'Andrew Bynum', 'Tyson Chandler', 'Dirk Nowitzki', 'Rajon Rondo',
       'Derrick Rose', "Amar'e Stoudemire", 'Al Horford', 'Za

In [6]:
#read in data from database for season stats table
seasons_stats_sql = 'SELECT * FROM public.seasons_stats'
season_stats_df = pd.read_sql(seasons_stats_sql, con=connection)
#divid year into season start and end
season_stats_df=season_stats_df.rename(columns = {'Year':'Year_season_end'})
season_stats_df['Year_season_end'] = pd.to_numeric(season_stats_df['Year_season_end'])
season_stats_df['Year_season_start'] = (season_stats_df['Year_season_end'] - 1)
season_stats_df['Player'] = season_stats_df['Player'].apply(lambda row: row.replace("*",""))
season_stats_df.head()


,Year_season_end,Player,Pos,Age,Tm,G,GS,MP,PER,TS_pct,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year_season_start
0,1978,Zaid Abdul-Aziz,PF-C,31,TOT,16,0,158.0,11.4,0.449,...,19.0,31.0,50.0,10.0,3.0,3.0,14.0,29.0,63.0,1977
1,1978,Zaid Abdul-Aziz,C,31,BOS,2,0,24.0,11.5,0.279,...,6.0,9.0,15.0,3.0,1.0,1.0,3.0,4.0,8.0,1977
2,1978,Mel Bennett,PF,23,IND,31,0,285.0,7.6,0.367,...,49.0,44.0,93.0,22.0,11.0,7.0,30.0,54.0,74.0,1977
3,1978,Mack Calvin,PG,30,DEN,77,0,988.0,14.0,0.551,...,11.0,73.0,84.0,148.0,46.0,5.0,108.0,87.0,467.0,1977
4,1978,Austin Carr,SG,29,CLE,82,0,2186.0,13.6,0.484,...,76.0,111.0,187.0,225.0,68.0,19.0,146.0,168.0,1011.0,1977


In [7]:
# get list of every nba player
AllPlayersList = season_stats_df.Player.unique()
AllPlayersList_df = pd.DataFrame(AllPlayersList)
#AllPlayersList_df.to_csv('listofplayers_test.csv')

#check and make sure every all-nba player shows up in the season data dataset
#players who weren't in the league in 2017 will show up here, that's okay!
#I cleaned up a bunch of names in the excel
for player in AllNBAPlayersList:
    if player not in AllPlayersList:   
        print(player)

Luka Doncic
Jayson Tatum
Ben Simmons


In [8]:
#merge allNBA status into season stats
Season_Stats_ML_DF = pd.merge(season_stats_df,All_NBA_Unpivoted_df,how='left',left_on = ['Player','Year_season_start'], right_on = ['PlayerName_Formatted','Season_Start_Year'])

In [9]:
# get rid of duplicate cols
Season_Stats_ML_DF = Season_Stats_ML_DF.drop(['Season_Start_Year','PlayerName_Formatted'],axis=1)

In [10]:
# fill in zeros for players that arent all nba
Season_Stats_ML_DF['AllNBA_Center'] = Season_Stats_ML_DF['AllNBA_Center'].fillna(int(0))
Season_Stats_ML_DF['AllNBA_Forward'] = Season_Stats_ML_DF['AllNBA_Forward'].fillna(int(0))
Season_Stats_ML_DF['AllNBA_Guard'] = Season_Stats_ML_DF['AllNBA_Guard'].fillna(int(0))
Season_Stats_ML_DF['Tm_1st'] = Season_Stats_ML_DF['Tm_1st'].fillna(int(0))
Season_Stats_ML_DF['Tm_2nd'] = Season_Stats_ML_DF['Tm_2nd'].fillna(int(0))
Season_Stats_ML_DF['Tm_3rd'] = Season_Stats_ML_DF['Tm_3rd'].fillna(int(0))

In [11]:
#get list of positions
Positions_List = Season_Stats_ML_DF['Pos'].unique()
Positions_List

array(['PF-C', 'C', 'PF', 'PG', 'SG', 'SF', 'SF-SG', 'SG-SF', 'C-PF',
       'SF-PF', 'PG-SG', 'SG-PG', 'PF-SF', 'PG-SF', 'SG-PF', 'C-SF'],
      dtype=object)

In [12]:
# create functions to determine if player is a guard forward and/or center
def is_forward(pos):
    possible_pos = [ 'SF',  'PF', 'PF-C', 'SG-SF', 'PF-SF', 'SF-SG', 'SF-PF', 'C-PF', 'SG-PF', 'C-SF', 'PG-SF']
    if pos in possible_pos: isF = 1
    else: isF = 0
    return isF

def is_guard(pos):
    possible_pos = ['PG','SG','SG-SF','PG-SG','SG-PG','SF-SG','SG-PF','PG-SF']
    if pos in possible_pos: isG = 1
    else: isG = 0
    return isG

def is_center(pos):
    possible_pos = ['C','PF-C','C-PF','C-SF']
    if pos in possible_pos: isC = 1
    else: isC = 0
    return isC

In [13]:
#Create dummy columns for G, F, C
Season_Stats_ML_DF['is_Guard'] = Season_Stats_ML_DF['Pos'].apply(lambda pos: is_guard(pos) )
Season_Stats_ML_DF['is_Forward'] = Season_Stats_ML_DF['Pos'].apply(lambda pos: is_forward(pos) )
Season_Stats_ML_DF['is_Center'] = Season_Stats_ML_DF['Pos'].apply(lambda pos: is_center(pos) )

In [14]:
#Drop the Pos Column since we have dummy columns
Season_Stats_ML_DF=Season_Stats_ML_DF.drop(['Pos'],axis=1)

In [15]:
# drop year  and player name
Season_Stats_ML_DF=Season_Stats_ML_DF.drop(['Year_season_end','Player'],axis=1)
Season_Stats_ML_DF=Season_Stats_ML_DF.drop(['Year_season_start'],axis=1)

Season_Stats_ML_DF.columns

Index(['Age', 'Tm', 'G', 'GS', 'MP', 'PER', 'TS_pct', '3PAr', 'FTr', 'ORB_pct',
       'DRB_pct', 'TRB_pct', 'AST_pct', 'STL_pct', 'BLK_pct', 'TOV_pct',
       'USG_pct', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP',
       'FG', 'FGA', 'FG_pct', '3P', '3PA', '3P_pct', '2P', '2PA', '2P_pct',
       'eFG_pct', 'FT', 'FTA', 'FT_pct', 'ORB', 'DRB', 'TRB', 'AST', 'STL',
       'BLK', 'TOV', 'PF', 'PTS', 'AllNBA_Center', 'AllNBA_Forward',
       'AllNBA_Guard', 'Tm_1st', 'Tm_2nd', 'Tm_3rd', 'is_Guard', 'is_Forward',
       'is_Center'],
      dtype='object')

In [16]:
#Creating Per Game Statistics

#Points per Game
Season_Stats_ML_DF["PPG"] = Season_Stats_ML_DF["PTS"]/Season_Stats_ML_DF["G"]
#Assits Per Game
Season_Stats_ML_DF["APG"] = Season_Stats_ML_DF["AST"]/Season_Stats_ML_DF["G"]
#Rebounds Per Game
Season_Stats_ML_DF["RPG"] = Season_Stats_ML_DF["TRB"]/Season_Stats_ML_DF["G"]
#Blocks Per Game
Season_Stats_ML_DF["RPG"] = Season_Stats_ML_DF["BLK"]/Season_Stats_ML_DF["G"]
#STLs Per Game
Season_Stats_ML_DF["SPG"] = Season_Stats_ML_DF["STL"]/Season_Stats_ML_DF["G"]

In [17]:
Season_Stats_ML_DF['3PAr'] = Season_Stats_ML_DF['3PAr'].fillna(int(0))
Season_Stats_ML_DF['3P'] = Season_Stats_ML_DF['3P'].fillna(int(0))
Season_Stats_ML_DF['3PA'] = Season_Stats_ML_DF['3PA'].fillna(int(0))
Season_Stats_ML_DF['3P_pct'] = Season_Stats_ML_DF['3P_pct'].fillna(int(0))


In [18]:
#make dummies for team
#Season_Stats_ML_DF = pd.get_dummies(Season_Stats_ML_DF,columns=['Tm'])
#I don't like this lol

In [19]:
# Fill null values with zero
for col in list(Season_Stats_ML_DF.columns):
    Season_Stats_ML_DF[col] = Season_Stats_ML_DF[col].fillna(int(0))

ALL NBA CENTER MACHINE LEARNING SECTION

In [20]:
#Create Dataset for X & y, make sure it is only centers...we don't need guard or forward data her

X = Season_Stats_ML_DF[Season_Stats_ML_DF.is_Center==1]
X = X.drop(['AllNBA_Center','AllNBA_Forward','AllNBA_Guard','is_Guard','is_Center','is_Forward'], axis=1)
X = X.drop(['Tm_1st','Tm_2nd','Tm_3rd'], axis=1)
#I dont think team is that important
X = X.drop(['Tm'], axis=1)
y = Season_Stats_ML_DF[Season_Stats_ML_DF.is_Center==1]
y = y["AllNBA_Center"].values.reshape(-1, 1)

# Splitting into Train and Test sets
# Creating StandardScaler instance
# Fitting Standard Scaller
# Scaling data

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

scaler = StandardScaler()
X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

from sklearn import tree
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


# Creating the decision tree classifier instance
model = tree.DecisionTreeClassifier()

model = model.fit(X_train_scaled, y_train)

# Making predictions using the testing data
predictions = model.predict(X_test_scaled)

# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,951,11
Actual 1,15,10


Accuracy Score : 0.9736575481256332
Classification Report
              precision    recall  f1-score   support

         0.0       0.98      0.99      0.99       962
         1.0       0.48      0.40      0.43        25

    accuracy                           0.97       987
   macro avg       0.73      0.69      0.71       987
weighted avg       0.97      0.97      0.97       987



In [21]:
# We can sort the features by their importance.
model  = model .fit(X_train_scaled, y_train)
importances =model.feature_importances_
sorted(zip(model.feature_importances_, X.columns), reverse=True)

[(0.3965565553631468, 'WS'),
 (0.10424158600787564, 'PPG'),
 (0.07075611177830947, 'DWS'),
 (0.053329109337721094, 'BLK_pct'),
 (0.0511639741057416, 'Age'),
 (0.045558808276167186, 'WS/48'),
 (0.044388019112146765, 'TS_pct'),
 (0.026638935353875907, 'USG_pct'),
 (0.02178259380764601, 'STL'),
 (0.019516152051381696, 'PER'),
 (0.019059769581690256, 'PF'),
 (0.017358004440467912, '3PAr'),
 (0.01667729838397898, 'VORP'),
 (0.016598796104862524, 'FTA'),
 (0.013076801049712107, '2P_pct'),
 (0.011435861749014157, 'DRB'),
 (0.01137751551560082, '2P'),
 (0.011363640496679359, 'ORB_pct'),
 (0.010891296903823004, 'GS'),
 (0.00989641882126224, 'DBPM'),
 (0.009802167213440701, 'OBPM'),
 (0.009189531762600659, 'STL_pct'),
 (0.005245140288930714, 'MP'),
 (0.0037779186135135953, 'G'),
 (0.00031799388041077556, 'RPG'),
 (0.0, 'eFG_pct'),
 (0.0, 'TRB_pct'),
 (0.0, 'TRB'),
 (0.0, 'TOV_pct'),
 (0.0, 'TOV'),
 (0.0, 'SPG'),
 (0.0, 'PTS'),
 (0.0, 'OWS'),
 (0.0, 'ORB'),
 (0.0, 'FTr'),
 (0.0, 'FT_pct'),
 (0.0,

ALL NBA GUARD MACHINE LEARNING SECTION

In [22]:
#Create Dataset for X & y, make sure it is only guard...we don't need center or forward data her

X = Season_Stats_ML_DF[Season_Stats_ML_DF.is_Guard==1]
X = X.drop(['AllNBA_Center','AllNBA_Forward','AllNBA_Guard','is_Guard','is_Center','is_Forward'], axis=1)
X = X.drop(['Tm_1st','Tm_2nd','Tm_3rd'], axis=1)
#I dont think team is that important
X = X.drop(['Tm'], axis=1)
y = Season_Stats_ML_DF[Season_Stats_ML_DF.is_Guard==1]
y = y["AllNBA_Guard"].values.reshape(-1, 1)

# Splitting into Train and Test sets
# Creating StandardScaler instance
# Fitting Standard Scaller
# Scaling data

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

scaler = StandardScaler()
X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


# Creating the decision tree classifier instance
model = tree.DecisionTreeClassifier()

model = model.fit(X_train_scaled, y_train)

# Making predictions using the testing data
predictions = model.predict(X_test_scaled)

# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,1881,20
Actual 1,21,40


Accuracy Score : 0.9791029561671764
Classification Report
              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99      1901
         1.0       0.67      0.66      0.66        61

    accuracy                           0.98      1962
   macro avg       0.83      0.82      0.83      1962
weighted avg       0.98      0.98      0.98      1962



In [23]:
# We can sort the features by their importance.
model  = model .fit(X_train_scaled, y_train)
importances =model.feature_importances_
sorted(zip(model.feature_importances_, X.columns), reverse=True)

[(0.4594190117872336, 'WS'),
 (0.07526764445295789, 'PER'),
 (0.05560728437565184, 'FT'),
 (0.04340657602649102, 'PPG'),
 (0.031072084884609434, '2P_pct'),
 (0.02939706542994553, 'FG'),
 (0.027692392503412086, 'Age'),
 (0.02554076844404576, 'FTr'),
 (0.02118238539814103, 'DWS'),
 (0.02008377975503651, 'APG'),
 (0.01966821695363252, 'BLK_pct'),
 (0.019063041047366894, 'OBPM'),
 (0.018845795481015597, 'AST'),
 (0.01594397129047285, 'FT_pct'),
 (0.0156911015696393, '3P_pct'),
 (0.01229639359923572, 'ORB'),
 (0.011588216473084443, 'AST_pct'),
 (0.011535482311192979, 'STL'),
 (0.011333944503646125, 'DRB'),
 (0.010070127080259856, 'MP'),
 (0.008930730826869125, 'OWS'),
 (0.006424703004891681, 'PTS'),
 (0.006157007046354532, 'RPG'),
 (0.005990050793144254, '2PA'),
 (0.005496497934643184, 'eFG_pct'),
 (0.004283135336594454, 'VORP'),
 (0.004283135336594454, 'TRB'),
 (0.004283135336594454, 'DBPM'),
 (0.00404806093207438, 'TOV'),
 (0.0032123515024458406, 'PF'),
 (0.0032123515024458406, 'G'),
 (0.

ALL NBA FORWARD MACHINE LEARNING SECTION

In [24]:
#Create Dataset for X & y, make sure it is only guard...we don't need center or forward data her

X = Season_Stats_ML_DF[Season_Stats_ML_DF.is_Forward==1]
X = X.drop(['AllNBA_Center','AllNBA_Forward','AllNBA_Guard','is_Guard','is_Center','is_Forward'], axis=1)
X = X.drop(['Tm_1st','Tm_2nd','Tm_3rd'], axis=1)
#I dont think team is that important
X = X.drop(['Tm'], axis=1)
y = Season_Stats_ML_DF[Season_Stats_ML_DF.is_Forward==1]
y = y["AllNBA_Forward"].values.reshape(-1, 1)

# Splitting into Train and Test sets
# Creating StandardScaler instance
# Fitting Standard Scaller
# Scaling data

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

scaler = StandardScaler()
X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


# Creating the decision tree classifier instance
model = tree.DecisionTreeClassifier()

model = model.fit(X_train_scaled, y_train)

# Making predictions using the testing data
predictions = model.predict(X_test_scaled)

# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,1931,12
Actual 1,17,31


Accuracy Score : 0.9854344550477148
Classification Report
              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99      1943
         1.0       0.72      0.65      0.68        48

    accuracy                           0.99      1991
   macro avg       0.86      0.82      0.84      1991
weighted avg       0.98      0.99      0.99      1991



In [25]:
# We can sort the features by their importance.
model  = model .fit(X_train_scaled, y_train)
importances =model.feature_importances_
sorted(zip(model.feature_importances_, X.columns), reverse=True)

[(0.47745702887321834, 'WS'),
 (0.07891367522270322, 'USG_pct'),
 (0.0559487501776141, 'PPG'),
 (0.05471273438491312, 'DWS'),
 (0.04485603196237916, 'DRB'),
 (0.03996413389871728, 'WS/48'),
 (0.02287423196238931, 'APG'),
 (0.01966282547366896, 'eFG_pct'),
 (0.018116241232527627, '2PA'),
 (0.017353703406778654, 'SPG'),
 (0.016340093695018043, 'RPG'),
 (0.015852229298776426, 'VORP'),
 (0.014983323326058081, 'TRB'),
 (0.011213872072462905, 'ORB_pct'),
 (0.010840076336714142, 'FG_pct'),
 (0.00983400453999644, 'DRB_pct'),
 (0.00936613178779572, '2P_pct'),
 (0.006541425375603362, 'TRB_pct'),
 (0.0064726660486924235, 'AST'),
 (0.006343200364221442, 'Age'),
 (0.006308741283068992, 'AST_pct'),
 (0.006105330350563138, 'FT'),
 (0.005966938194010363, 'BPM'),
 (0.005946750341457602, 'GS'),
 (0.005678320638544583, 'G'),
 (0.005527695712132662, 'STL'),
 (0.005233140300482688, 'TOV'),
 (0.004360950250402241, 'MP'),
 (0.004360950250402241, 'FTA'),
 (0.003270712687801681, 'PER'),
 (0.003270712687801681,